In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos token

In [2]:
from peft import get_peft_model, LoraConfig, TaskType

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

/opt/homebrew/Caskroom/miniconda/base/envs/llm-lab/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [3]:
from datasets import load_dataset

dataset = load_dataset("mlabonne/guanaco-llama2-1k")

def tokenize_function(examples):
    tokens = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()  # Copy input_ids to labels for training
    return tokens

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

In [4]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    remove_unused_columns=False,

    save_strategy="steps",
    save_steps=1000,
    save_total_limit=3,

    logging_strategy="steps",
    logging_steps=200,

    lr_scheduler_type="cosine",
    warmup_steps=200,

    gradient_accumulation_steps=4,  # optional
)

In [5]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],  # Replace with your training dataset
    eval_dataset=None,   # Replace with your evaluation dataset
    tokenizer=tokenizer,
)

/var/folders/wk/q4v0g9_d3_l5g95w3jgn_x6h0000gn/T/ipykernel_74845/3759506436.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [6]:
trainer.train()

/opt/homebrew/Caskroom/miniconda/base/envs/llm-lab/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
200,3.698900
400,2.264200
600,2.208500
800,2.184400
1000,2.175200
1200,2.190500


/opt/homebrew/Caskroom/miniconda/base/envs/llm-lab/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=1250, training_loss=2.4382077758789062, metrics={'train_runtime': 22369.3553, 'train_samples_per_second': 0.447, 'train_steps_per_second': 0.056, 'total_flos': 2621980016640000.0, 'train_loss': 2.4382077758789062, 'epoch': 10.0})

In [7]:
model.save_pretrained("./results/lora_finetuned_model")
tokenizer.save_pretrained("./results/lora_finetuned_model")

('./results/lora_finetuned_model/tokenizer_config.json',
 './results/lora_finetuned_model/special_tokens_map.json',
 './results/lora_finetuned_model/vocab.json',
 './results/lora_finetuned_model/merges.txt',
 './results/lora_finetuned_model/added_tokens.json',
 './results/lora_finetuned_model/tokenizer.json')